# Using GPS to measure True Air Velocity for calibrating airspeed and heading

In 1998 Doug Gray wrote [a short paper](https://www.kilohotel.com/rv8/rvlinks/doug_gray/TAS_FNL4.pdf) describing a method for finding true air velocity (airspeed and heading) from GPS velocity (groundspeed and ground track). The test flight is simple: record ground velocity while straight and level at the same airspeed and altitude, for three directions (ideally, 90–120° apart).

Although his spreadsheet formulas work, it's a big jump from the diagram to the formulas. What follows is a re-derived explanation, and Python code you can use to run calculations for yourself. There is also [a spreadsheet](https://docs.google.com/spreadsheets/d/1_gO5Yt-bQr3e1e3rRNqG1UHT8yLdF9CCd2a2QoJ2SFY/edit?gid=712118661#gid=712118661) which you can duplicate for your own use.

## Derivation
![Diagram](Diagram.svg)

Begin by recording the ground velocity (speed and track) for three different compass headings (e.g. from your GPS). Fly at the same airspeed and altitude. 120° apart is good, but not required. Although not necessary, it's also prudent to record the headings, pressure altitude, outside air temperature, and wind forecast. Call these three ground velocities $g_1, g_2, $ and $g_3$.

When these ground vectors are placed head-to-head at point $D$, a circle can be defined by the tails of these vectors ($A, B, C$). The air velocity $a_1$ is the familiar wind triangle, i.e.
$$a_1=g_1-w$$
And similarly for $a_2$ and $a_3$. TAS is the magnitude of any and all of these vectors (you did fly the same airspeed didn't you?), and it is also the radius of the circle. If we can find the center of the circle, we can calculate all the interesting variables.

We can find the center of the circle with a few key identities. $p_1$ and $p_2$ are chords of the circle, so their perpendicular bisectors $q_1$ and $q_2$ intersect at the center of the circle. The perpendicular slope is the negative inverse of the slope:
$$m'=-1/m$$
The intersection of two lines in point-slope form is:
$$\left(\frac{b_2-b_1}{m_1-m_2}, m_1\frac{b_2-b_1}{m_1-m_2} + b_1\right)$$

The y-intercept can be found from the slope of $q$ and midpoint of $p$. If we declare point $A=(0,0)$, the equations are quite neat.

$$
\begin{align*}
p_1 &= g_1-g_2 &
p_2 &= g_1-g_3 \\
m_{q_1} &= -x_{p_1}/y_{p_1} &
m_{q_2} &= -x_{p_2}/y_{p_2} \\
b_{q_1} &= \frac{y_{p_1}-m_{q_1}x_{p_1}}{2} &
b_{q_2} &= \frac{y_{p_2}-m_{q_2}x_{p_2}}{2}
\end{align*}
$$
And now we can calculate the vectors of interest:
$$
\begin{align*}
a_1 &= \left(\frac{b_{q_2}-b_{q_1}}{m_{q_1}-m_{q_2}}, 
 m_{q_1} \frac{b_{q_2}-b_{q_1}}{m_{q_1}-m_{q_2}} + b_{q_1}\right) \\
w &= g_1-a_1 \\
a_2 &= g_2-w \\
a_3 &= g_3-w 
\end{align*}
$$

## Code
If you [open this as an interactive binder on mybinder.org](https://mybinder.org/v2/gh/fugalh/TAS/HEAD?labpath=TAS.ipynb), you can fill `data` with your own data points and execute the code to get your results.

In [1]:
# flight test data (groundspeed, ground track in degrees)
data = [
        (140, 192),
        (112, 283),
        (120,  20),
]

In [2]:
# preamble
import numpy as np

def cartesian(rho, theta):
    x = rho * np.cos(theta)
    y = rho * np.sin(theta)
    return x, y

def polar(v):
    x, y = v
    rho = np.hypot(x, y)
    theta = np.arctan2(y, x)
    return rho, theta
    
def fromCompass(speed, track):
    theta = np.radians(90 - track)
    return cartesian(speed, theta)

def toCompass(v):
    speed, theta = polar(v)
    track = (90 - np.degrees(theta)) % 360
    return speed, track

def mag(v):
    return np.sqrt(v[0]**2 + v[1]**2)

# The interesting bit
def calculate(data):
        g1, g2, g3 = (fromCompass(speed, track) for speed, track in data)
        
        # Chords
        p1 = np.subtract(g1, g2)
        p2 = np.subtract(g1, g3)
        
        # Bisectors
        mq1 = -p1[0]/p1[1]
        mq2 = -p2[0]/p2[1]
        bq1 = (p1[1] - mq1 * p1[0]) / 2
        bq2 = (p2[1] - mq2 * p2[0]) / 2
        
        # Circle center
        xO = (bq2-bq1)/(mq1-mq2)
        yO = mq1 * xO + bq1

        # Results
        a1 = [xO, yO]
        w = np.subtract(g1, a1)
        a2 = np.subtract(g2, w)
        a3 = np.subtract(g3, w)
        tas = mag(a1)
        headings = [toCompass(v)[1] for v in [a1, a2, a3]]
        
        return tas, w, headings

In [3]:
# print results
tas, w, headings = calculate(data)
windSpeed, windFrom = toCompass(-w)

from IPython.display import Markdown
Markdown(f"""Gray's results:
- TAS: 130
- Headings:
  - 200°
  - 287.8°
  - 11.7°
- Wind: 314.8° at 20.6
   
Our results:
- TAS: {tas:.0f}
- Headings:
  - {headings[0]:.1f}°
  - {headings[1]:.1f}°
  - {headings[2]:.1f}°
- Wind: {windFrom:.1f}° at {windSpeed:.1f}""")

Gray's results:
- TAS: 130
- Headings:
  - 200°
  - 287.8°
  - 11.7°
- Wind: 314.8° at 20.6
   
Our results:
- TAS: 130
- Headings:
  - 199.7°
  - 287.8°
  - 11.7°
- Wind: 314.8° at 20.6

## Appendix
Compass angles increase clockwise, but in cartesian coordinates they usually go counter-clockwise. North is 0° on the compass, but 0° in cartesian coordinates is usually along the positive x axis. For this reason we convert from compass to cartesian angles as $\phi'=90-\phi$, just so that it's easier to reason about values when debugging.

Conversion from a polar vector $v$ to cartesian coordinates $(x,y)$:
$$
\begin{align*}
\rho &= |v| \\
\theta &= \angle v \\
(x, y) &= (\rho\sin(\theta), \rho\cos(\theta))
\end{align*}
$$

And from cartesian to polar (when programming, use `atan2` to automatically get the right quadrant from $\arctan$):
$$
(\rho, \theta) = (\sqrt{x^2 + y^2}, \arctan(y/x))
$$

And you may need to remember radians and degrees:
$$
\begin{align*}
\textrm{radians}&=\textrm{degrees}\frac{\pi}{180} &
\textrm{degrees}&=\textrm{radians}\frac{180}{\pi}
\end{align*}
$$

And of course don't forget that true airspeed is not calibrated airspeed is not indicated airspeed. But you knew that, that's why you were doing this flight test in the first place…

## References
- [Using GPS to accurately establish True Airspeed (TAS)](https://www.kilohotel.com/rv8/rvlinks/doug_gray/TAS_FNL4.pdf), Doug Gray, 1998
- [Flight Testing: Finding TAS from GPS Data](https://www.kitplanes.com/flight-testing-finding-tas-from-gps-data/), Kevin Horton, 2009